In [10]:
using KitBase, Plots, JLD2, Distributions, LinearAlgebra, Flux, KitBase.WriteVTK
using Flux: onecold, @epochs

In [4]:
function regime_data(ks, w, prim, swx, swy, f)
    Mu, Mv, Mxi, _, _1 = gauss_moments(prim, ks.gas.K)
    a = pdf_slope(prim, swx, ks.gas.K)
    b = pdf_slope(prim, swy, ks.gas.K)
    swt = -prim[1] .* (moments_conserve_slope(a, Mu, Mv, Mxi, 1, 0) .+ moments_conserve_slope(b, Mu, Mv, Mxi, 0, 1))
    A = pdf_slope(prim, swt, ks.gas.K)
    tau = vhs_collision_time(prim, ks.gas.μᵣ, ks.gas.ω)
    
    fr = chapman_enskog(ks.vs.u, ks.vs.v, prim, a, b, A, tau)
    L = norm((f .- fr) ./ prim[1])

    sw = (swx.^2 + swy.^2).^0.5
    x = [w; sw; tau]
    y = ifelse(L <= 0.005, [1.0, 0.0], [0.0, 1.0])

    return x, y
end

function regime_number(Y, rg=1)
   idx = 0
    for i in axes(Y, 2)
       if Y[1, i] == rg
            idx += 1
        end
    end
    println("NS regime: $(idx) of $(size(Y, 2))")
    return nothing
end

function accuracy(nn, X, Y)
    Y1 = nn(X)

    YA1 = [onecold(Y1[:, i]) for i in axes(Y1, 2)]
    YA = [onecold(Y[:, i]) for i in axes(Y, 2)]

    accuracy = 0.0
    for i in eachindex(YA)
        if YA[i] == YA1[i]
            accuracy += 1.0
        end
    end
    accuracy /= length(YA)

    return accuracy
end

accuracy (generic function with 1 method)

In [5]:
file = open("/home2/vavrines/Coding/KiT-RT/result/logs/test_a10_ev5.csv")
data = []
for line in eachline(file)
    a = split(line, ",")
    b = [parse(Float64, a[i]) for i = 2:length(a)]
    push!(data, b)
end

In [6]:
data[1] |> minimum

-4.998084560095

In [7]:
data[2]

6400-element Vector{Float64}:
  0.002932191272
  0.008792052591
  0.014638357201
  0.020462090525
  0.02625427279
  0.032005972868
  0.037708322056
  0.043352527745
  0.048929886978
  0.054431799872
  0.059849782875
  0.065175481852
  0.07040068496
  ⋮
 -2.181630817797
 -2.003362722442
 -1.822005586975
 -1.637839050954
 -1.45114708583
 -1.262217557081
 -1.071341780347
 -0.878814072239
 -0.68493129653
 -0.489992406406
 -0.294297983507
 -0.098149774448

In [8]:
data[4] |> minimum

0.014050650464

In [11]:
u = data[1]
v = data[2]

6400-element Vector{Float64}:
  0.002932191272
  0.008792052591
  0.014638357201
  0.020462090525
  0.02625427279
  0.032005972868
  0.037708322056
  0.043352527745
  0.048929886978
  0.054431799872
  0.059849782875
  0.065175481852
  0.07040068496
  ⋮
 -2.181630817797
 -2.003362722442
 -1.822005586975
 -1.637839050954
 -1.45114708583
 -1.262217557081
 -1.071341780347
 -0.878814072239
 -0.68493129653
 -0.489992406406
 -0.294297983507
 -0.098149774448

In [12]:
vtk_write_array("test.vtk", (u, v), ("u", "v"))

LoadError: DimensionMismatch("input should be a 2D or 3D array.")